<a href="https://colab.research.google.com/github/AshwinUnnikrishnan/Solutions/blob/master/Swedish_Lighthouses_Challenge_%5B_Ashwin_Unnikrishnan_%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Swedish Lighthouses Challenge


Skydio is scoping out a project to aid the search and rescue operations of the Swedish Coast Guard. Their goal is to establish a network of docked drones along the coastline that provide a rapid response to emergency transmissions.

(The Skydio Dock is a cloud-connected, weatherproof home base and charging station that enables our drones to fly without a pilot on-scene.)

The customer is interested in establishing coverage from their main office in Stockholm east to the island of Vindö and then southwest to Torö.

They have permission to build a dock into any lighthouse in the region. They are interested in building a route such that a drone can hop from dock to dock from Stockholm to Torö, so that they can recover their fleet and swap drones without sending humans to the lighthouses.

**Submission** - Make a copy of this notebook and fill out the code and writeup. Feel free to download and run locally if helpful. Share the completed Google Colab link with us by setting the permission to *Anyone with the link can view*.

**Guidelines** - This challenge tests the ability to write quality code, interact with APIs, visualize data, and employ algorithms to solve problems. We score submissions by the clarity of the code, visuals, and written documentation. We do not score by the amount of time spent, so go as deep as you like, or if you are short on time then take a simpler approach to Part 2 and 3.

<img width="500px" src="https://i.imgur.com/5H88NQ3.png"/>

# Part 1: Find all of the lighthouses

Use [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap) to find known lighthouses in the region and get that data into your notebook. OSM provides APIs to query their geographic database by structure type. Plot the lighthouses on a map using [pydeck](https://deckgl.readthedocs.io/en/latest/).

Note that you are free to install python packages in colab: `!pip install pydeck`

### Learning Overpass


Info about how to query in turbo [Turbo Overpass](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL)

How to use tags in Nodes : [Map_features](https://wiki.openstreetmap.org/wiki/Map_features)

### Lighthouse identification summary

For finding lighthouses we are taking the 3 major points(lat and lon) as input from user.


*   The code is written in such a way that given any set of points it will find the lighthouses covering all those points in a bounding box. The code can be reused.
* Corner case that I am not considering for a generic solution is, where the route is more around the land rather than the shortest line between two destination. Example diagram is shown in the link .[CornerCase](https://drive.google.com/file/d/18cX4yR_VH7GpOKiax9Qm94F6lwWtrWtT/view?usp=sharing)
* Considering the ground speed and battery time to increase the range of corner points to include more lighthouses to select from.
      Example : Stockholm being the west most point, I am taking a longitude
      that is 0.34 degree(20kms calculated using battery life of 28 mins( 
      reducing 2 mins for take off and landing and other corner cases )
      and ground speed of 12 mts/second.

### Installing pydeck and other tools

In [1]:
!pip install pydeck --quiet

     |████████████████████████████████| 4.3 MB 4.1 MB/s 
     |████████████████████████████████| 132 kB 45.1 MB/s 
     |████████████████████████████████| 793 kB 43.0 MB/s 
     |████████████████████████████████| 423 kB 55.0 MB/s 
     |████████████████████████████████| 132 kB 41.7 MB/s 
     |████████████████████████████████| 381 kB 55.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.6.6 requires traitlets>=5.2.2, but you have traitlets 5.1.1 which is incompatible.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.30 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.15.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.34.0 which is incompatible.
google-colab 1.0.0 requires tornado~=5.1.0, but you have tornado 6.2 whi

In [2]:
!pip install osmpythontools --quiet

### Importing pydeck to plot the lighthouses on a

In [3]:
import pandas as pd
import pydeck as pdk
from OSMPythonTools.overpass import Overpass

In [4]:
overpass = Overpass()

### Taking input about the locations the drone should visit

Get input lat and lon list of places to visit, by default this is taken as Stockholm, Vindo and Toro

In [5]:
#noOfPlaces = int(input("Enter total number of places : "))

In [6]:
'''placeList = []                #Stores a set containing lat and lon for each of the place
for i in range(noOfPlaces):
  lat = float(input("Enter {0} destination lat value ".format(str(i))))
  lon = float(input("Enter {0} destination lon value ".format(str(i))))
  placeList.append((lat,lon))
'''

'placeList = []                #Stores a set containing lat and lon for each of the place\nfor i in range(noOfPlaces):\n  lat = float(input("Enter {0} destination lat value ".format(str(i))))\n  lon = float(input("Enter {0} destination lon value ".format(str(i))))\n  placeList.append((lat,lon))\n'

Assigning Default Values for now

In [7]:
placeList = [(59.328982, 18.068531), (59.3763143, 18.6734587), (58.834266, 17.840492)]
placeList

[(59.328982, 18.068531), (59.3763143, 18.6734587), (58.834266, 17.840492)]

### Calculating the BBox boundaries



In [8]:
def getBoundingBox(placeLists, droneFlightTime = 28):
  '''
  Based on the input of locations we find the end corners north, south, east, west
  to find the bounding box inside which we need to find the lighthouses.
  Extra degrees are added to max, considering the drone flight time.
  Assuming the flight time as 28 mins instead of 30, considering time to take 
  off and land.
  
  Input Parameters:
                    placeLists : List of places to consider
                    droneFlightTime (int): Air time of the drone

  Returns:  south, west, north, east : The bounding box lat and lng in all four 
                                       direction.
  '''
  north, south, east, west = None, None, None, None
  for lat, lon in placeLists:
    if south == None or lat < south:
      south = lat
    if north == None or lat > north:
      north = lat
    if west == None or lon < west:
      west = lon
    if east == None or lon > east:       #can combine with above as if else
      east = lon

    #Add extra coverage as drone can fly out 28 mins in any direction by default
    #Ground Speed of Drone 12mts/second
    #additional lat and lng it can travel = 0.18 and 0.34 degree respectively
    # Values added here can change based on the speed and flight time and we can
    # write function to calculate the degree
  south -= 0.18
  west -= 0.34
  north += 0.18
  east += 0.34
    
  return south, west, north, east
    

In [9]:
south, west, north, east = getBoundingBox(placeList)

### Getting the light house data from OSM using overpass

Searching the OSM with the bounding box coordinates found for lighthouses

In [10]:
query = '''
(
  node["man_made"="lighthouse"]({0},{1},{2},{3});
);
out body;'''.format(south,west,north,east)

In [11]:
query

'\n(\n  node["man_made"="lighthouse"](58.654266,17.500492,59.5563143,19.0134587);\n);\nout body;'

In [12]:
res = overpass.query(query)

[overpass] downloading data: [timeout:25][out:json];
(
  node["man_made"="lighthouse"](58.654266,17.500492,59.5563143,19.0134587);
);
out body;


In [13]:
res.toJSON().keys()

dict_keys(['version', 'generator', 'osm3s', 'elements'])

In [14]:
#filtering the elements from the resultant of overpass query
filtered = [i for i in res.toJSON()['elements'] ]

In [15]:
# converting the result into a dataframe to easily handle
df = pd.DataFrame(filtered)

In [16]:
df.head()

,type,id,lat,lon,tags
0,node,358356239,59.320767,18.154908,"{'man_made': 'lighthouse', 'name': 'Blockhusud..."
1,node,799793245,59.256482,18.946644,"{'man_made': 'lighthouse', 'name': 'Österskär'..."
2,node,1029352189,59.182294,18.492252,"{'man_made': 'lighthouse', 'name': 'Östra Sten..."
3,node,1029397338,59.223811,18.618563,"{'man_made': 'lighthouse', 'name': 'Kofoten', ..."
4,node,1029397342,59.207756,18.565901,"{'man_made': 'lighthouse', 'name': 'Grönö', 's..."


In [17]:
df.keys()

Index(['type', 'id', 'lat', 'lon', 'tags'], dtype='object')

Taking the height of the lighthouse as we can plot a 3D map in pydeck
Some of the lighthouses have missing values for *seamark:light:1:height* so we assign 3 by default

In [18]:
listOfHeight = [ h['seamark:light:1:height'] if 'seamark:light:1:height' in h.keys() else 3 for h in df['tags'] ]

In [19]:
df['height'] = listOfHeight

In [20]:
df.head()

,type,id,lat,lon,tags,height
0,node,358356239,59.320767,18.154908,"{'man_made': 'lighthouse', 'name': 'Blockhusud...",6
1,node,799793245,59.256482,18.946644,"{'man_made': 'lighthouse', 'name': 'Österskär'...",7
2,node,1029352189,59.182294,18.492252,"{'man_made': 'lighthouse', 'name': 'Östra Sten...",3
3,node,1029397338,59.223811,18.618563,"{'man_made': 'lighthouse', 'name': 'Kofoten', ...",6
4,node,1029397342,59.207756,18.565901,"{'man_made': 'lighthouse', 'name': 'Grönö', 's...",7


### Creating the map with the data got from OSM

In [21]:
def drawMapBoxMap(df):
  view_state = pdk.ViewState(**{  
    "latitude": 59.196318798583725,
    "longitude": 18.599920729657768,
    "bearing": -1.45748987854251,
    "pitch": 0.5797101449275388,
    "zoom": 8,
  })


  column_layer = pdk.Layer(
      "ColumnLayer",
      data=df,
      get_position=["lon", "lat"],
      get_elevation="height",
      elevation_scale=100,
      radius=500,
      min_radius = 10,
      get_fill_color=[0, 0, 255, 140],
      pickable=True,
      auto_highlight=True,
      opacity=True,
      filled=True,
      radiusUnits='pixels',

  )
  MAPBOX_API_KEY = "pk.eyJ1IjoiaGlzdHJhdmVsc3RvcmllcyIsImEiOiJjbDZmYTRqa20wMWp3M2ltcTBkdWhudjdwIn0.TZUuKeVMFdSDZZtX_MPjzw"
  r = pdk.Deck(column_layer, 
             initial_view_state=view_state,
             api_keys={'mapbox' : MAPBOX_API_KEY},
             map_provider='mapbox',
            map_style='mapbox://styles/mapbox/streets-v11',         #Styles light, dark, streets, outdoors, imagery
  )
  resMapBoxMap = r.to_html(notebook_display=True, iframe_height=500, iframe_width=100)

**Result of Part 1** : Shows all the lighthouses

In [50]:
drawMapBoxMap(df)

<IPython.core.display.Javascript object>

Cleaning the dataframe to attributes that we are using

In [23]:
df = df[['lon', 'lat','height']]

In [24]:
df.head()

,lon,lat,height
0,18.154908,59.320767,6
1,18.946644,59.256482,7
2,18.492252,59.182294,3
3,18.618563,59.223811,6
4,18.565901,59.207756,7


# Part 2: Choose where to build docks

Suggest a minimal set of lighthouses at which to build a dock that would allow a drone to traverse from Stockholm to Torö, if each drone has a usable battery life of 30 minutes and travels at a ground speed of 12 meters per second. Plot the route, assuming approximate endpoints at Stockholm and Torö. Also print the total route distance.

It's okay to make reasonable assumptions here and state your thinking. You can go as straightforward or complex as you like.

### Calculating distance between two points in terms of kms

geopy helps locate the coordinates of addresses, cities, countries, and landmarks. Using here to calculate the distance between two coordinates

In [25]:
import geopy.distance

In [26]:
def findDistance(coordinate_1, coordinate_2):
  '''
    Calculates the straight line distance between two coordinates

    Input Parameters : coordinate_1, coordinat_2 ( tuples containing lat and lng
                       of the places)

    Returns : Straight line distance in kilometers

  '''
  return(geopy.distance.geodesic(coordinate_1, coordinate_2).km)


### Calculating the Lighthouses between each desatinations

In [27]:
route_1 = df[(df.lat >= 59.328982) & (df.lat <= (59.349003 + 0.18018)) & (df.lon >=  18.068531) & (df.lon <= (18.686806 + 0.34))] 
route_2 = df[(df.lat >= 58.834266) & (df.lat <= (59.349003 + 0.18018)) & (df.lon >=  17.840492) & (df.lon <= (18.686806 + 0.34))] 

In [52]:
drawMapBoxMap(route_1)

<IPython.core.display.Javascript object>

In [29]:
len(route_1)

49

* Checking straight line distance between Stockholm and the farthest lighthouse near Vindo we see that the distance is ~40kms, and we see from the above map that there are lots of lighthouses in the middle of the path and lot of outlier light houses which we can straight away remove from our list of light houses to consider for finding the path. Reducing the degree to the north of vindo to check for lighthouses.


* We can use Dynamic Programming to find the most efficient result with the resultant 49 lighthouses. But to store the distances we would require a big look up table. In case when this problem is to be made more generic, we should use the DP approach to find the minimal set. 

Here as we can visualize the data and get information from the data we try to reduce the memory and computation by simple visualization.

* Identified a lighthouse on the island of Vindo and removed all lighthouses on the right of it.

In [30]:
route_1_Updated = df[(df.lat >= 59.328982) & (df.lat <= (59.349003 + 0.03018)) & (df.lon >=  18.068531) & (df.lon <= (18.7))] 
len(route_1_Updated)
route_1_Updated = route_1_Updated.reset_index()
route_1_Updated = route_1_Updated.drop(['index'], axis=1)

We see that from 49 we have reduced to 15 lighthouses, it is not necessary to do this step, in case where we do not have any manual intervention we can come up with other optimization techniques like clustering to minimize the lighthouses to consider.

In [51]:
drawMapBoxMap(route_1_Updated)

<IPython.core.display.Javascript object>

In [53]:
drawMapBoxMap(route_2)

<IPython.core.display.Javascript object>

In [33]:
route_2_Updated = df[(df.lat >= 58.834266) & (df.lat <= (59.3763143)) & (df.lon >=  17.840492) & (df.lon <= (18.6734587 + 0.18))] 
route_2_Updated = route_2_Updated.reset_index()
route_2_Updated = route_2_Updated.drop(['index'], axis=1)
drawMapBoxMap(route_2_Updated)

<IPython.core.display.Javascript object>

### Calculating the optimal path between 

In [34]:
def intermediate(routePassage, point1, point2, path):
  '''
    Finds all the intermediate stops from point1 to point2

    Input Parameters : 
                        routePassage : Table that contains all the information of 
                                      stop to take for optimal distance.
                        point1 : source
                        point2 : destination
                        path : contains the sequence of stops from point1 to point2
  '''

  if routePassage[point1][point2] == -1:    # It is direct connection btw points
    path.append(point2)
    return
  else:
    x = routePassage[point1][point2]        # x is intermediate stop between points
    intermediate(routePassage, x, point2, path)
    #path.append(x)
    intermediate(routePassage, point1, x, path)

In [35]:
def calculateDistanceBetweenTwoDestinations(locations, src=None, dest=None):
  '''
    calculates the shortest distance between two points src and dest
    The function is also used to find the shortest distance between each points
    passed in the locations when src and dest in None.


    Input Parameters : 
                      locations : Contains a list of places that the drone can stop
                      src : Lists the place where the drone should start
                      dest : Lists the place where the drone should stop(end point)
                      
    Returns :  distance : The shortest distance between src and dest (kms)
               routeDf : List of places the drone would stop to charge( includes
                          src and dest )
               routeDistance : Table that contains the distances for each location
                                to every other location.
               routePassage : Table that contains the stops(battery charging) that
                              drone should make
  '''
  #adding source
  route = locations.copy(deep=True)
  if src != None:
    route.loc[-1] = [src[0], src[1], 30]
    route.index = route.index + 1
    route = route.sort_index()

  #adding destination
  if dest != None:
    route.loc[len(route)] = [dest[0], dest[1], 30]
    route = route.sort_index()

  #initialize all the routes to 0
  routeDistance = [[0 for i in range(len(route))] for j in range(len(route))]

  #initialize the hops if unreachable to -inf
  routePassage = [[float('-inf') for i in range(len(route))] for j in range(len(route))]

  print(route)
  # Finding all the nodes that are directly connected, drone flight distance of 20kms
  # This is the most optimal way from point A to point B because it is a straight
  #  line distance
  for i in range(0,len(route)):
    for j in range(i+1, len(route)):
      temp = findDistance((route.lat[i],route.lon[i]),(route.lat[j],route.lon[j]))
      if temp <= 20:
        routeDistance[i][j] = temp
        routeDistance[j][i] = temp
        routePassage[i][j] = -1
        routePassage[j][i] = -1

      else:
        routeDistance[i][j] = float('inf')
        routeDistance[j][i] = float('inf')
  
  
  # Filling the missing values from above result by taking intermediate steps
  # The steps is basically means intermediate stop to charge drone
  # and moving forward modified version of All Source Shortest Path

  flag = True
  while flag == True:
    flag = False
    for i in range(0,len(route)):
      for j in range(i+1, len(route)):
        if routeDistance[i][j] == float('inf'):
          flag = True
          for k in range(0,len(route)):
            if k != i and k != j and routeDistance[i][k] != float('inf') and routeDistance[i][k] != float('inf') and ( routeDistance[i][k] + routeDistance[k][j] < routeDistance[i][j]):
              routeDistance[i][j] = routeDistance[i][k] + routeDistance[k][j]
              routeDistance[j][i] = routeDistance[i][j]
              routePassage[i][j] = k
              routePassage[j][i] = k

  # The above algorithm can be further optimized

  # To find the shortest path and hop places between source and destination

  distance = routeDistance[0][len(route)-1]

  temp = len(route) - 1
  finalRoute = []
  
  intermediate(routePassage, 0, temp, finalRoute)
  
  finalRoute.append(0)
  finalRoute.reverse()
  
  routeDf = route.iloc[finalRoute]
  
  return distance, routeDf, routeDistance, routePassage

In [36]:
distance_1, finalRoute_1, routeDistance_1, routePassage_1 = calculateDistanceBetweenTwoDestinations(route_1_Updated, (18.068531, 59.328982), (18.6734587, 59.3763143))

          lon        lat height
0   18.068531  59.328982   30.0
1   18.452297  59.361799      6
2   18.415089  59.358584      6
3   18.389168  59.371421      5
4   18.376364  59.362923      4
5   18.335251  59.371713      5
6   18.294481  59.377660      4
7   18.310531  59.366465      6
8   18.279718  59.374118      7
9   18.268732  59.365634      3
10  18.259006  59.363986      6
11  18.176414  59.332282      6
12  18.605785  59.369791      7
13  18.209156  59.334586      7
14  18.614218  59.373879     16
15  18.673459  59.376314      5
16  18.673459  59.376314   30.0


In [37]:
print(finalRoute_1)
print(distance_1)

          lon        lat height
0   18.068531  59.328982   30.0
2   18.415089  59.358584      6
16  18.673459  59.376314   30.0
34.816947965156515


In [54]:
drawMapBoxMap(finalRoute_1)

<IPython.core.display.Javascript object>

In [39]:
distance_2, finalRoute_2, routeDistance_2, routePassage_2 = calculateDistanceBetweenTwoDestinations(route_2_Updated, (18.6734587, 59.3763143), (17.840492,58.834266))

          lon        lat height
0   18.673459  59.376314   30.0
1   18.154908  59.320767      6
2   18.492252  59.182294      3
3   18.618563  59.223811      6
4   18.565901  59.207756      7
5   18.613513  59.217802      7
6   18.015065  58.856961     13
7   18.452297  59.361799      6
8   18.415089  59.358584      6
9   18.389168  59.371421      5
10  18.376364  59.362923      4
11  18.335251  59.371713      5
12  18.310531  59.366465      6
13  18.279718  59.374118      7
14  18.268732  59.365634      3
15  18.259006  59.363986      6
16  18.176414  59.332282      6
17  18.810237  59.313307      3
18  18.810817  59.318295      8
19  18.747238  59.316970     12
20  18.759072  59.304925      6
21  18.830493  59.355928      5
22  18.753418  59.371849      7
23  18.744621  59.369672      3
24  18.605785  59.369791      7
25  18.429988  59.131701     24
26  18.422345  59.128426      5
27  18.407576  59.124412      3
28  18.426427  59.120652      4
29  18.715127  59.296403      6
30  18.6

In [40]:
print(distance_2)
print(finalRoute_2)

78.94201861421794
          lon        lat height
0   18.673459  59.376314   30.0
4   18.565901  59.207756      7
35  18.409850  59.113394      3
40  18.186326  58.978232      7
54  17.928323  58.877950      3
59  17.840492  58.834266   30.0


In [55]:
drawMapBoxMap(finalRoute_2)

<IPython.core.display.Javascript object>

### Visualizing the minimal set of lighthouses

In [42]:
finalRoute = pd.concat([finalRoute_1, finalRoute_2])

In [56]:
drawMapBoxMap(finalRoute)

<IPython.core.display.Javascript object>

In [44]:
finalRoute

,lon,lat,height
0,18.068531,59.328982,30.0
2,18.415089,59.358584,6
16,18.673459,59.376314,30.0
0,18.673459,59.376314,30.0
4,18.565901,59.207756,7
35,18.409850,59.113394,3
40,18.186326,58.978232,7
54,17.928323,58.877950,3
59,17.840492,58.834266,30.0


In [45]:
finalRoute = finalRoute.drop_duplicates()
finalRoute = finalRoute.reset_index()
finalRoute = finalRoute.drop(['index'], axis=1)
finalRoute

,lon,lat,height
0,18.068531,59.328982,30.0
1,18.415089,59.358584,6
2,18.673459,59.376314,30.0
3,18.565901,59.207756,7
4,18.409850,59.113394,3
5,18.186326,58.978232,7
6,17.928323,58.877950,3
7,17.840492,58.834266,30.0


In [46]:
distance, finalRoute, routeDistance, routePassage = calculateDistanceBetweenTwoDestinations(finalRoute)

         lon        lat height
0  18.068531  59.328982   30.0
1  18.415089  59.358584      6
2  18.673459  59.376314   30.0
3  18.565901  59.207756      7
4  18.409850  59.113394      3
5  18.186326  58.978232      7
6  17.928323  58.877950      3
7  17.840492  58.834266   30.0


### Distance In Total

Below output shows the distance from point1 to point2 in the cell in kilometers units

routeDistance[0][7] prints the longest the farthest drone has to travel

In [47]:
pd.DataFrame(routeDistance)

,0,1,2,3,4,5,6,7
0,0.000000,19.993434,34.816948,38.867196,52.658215,72.439730,91.029943,98.056666
1,19.993434,0.000000,14.823514,18.873762,32.664781,52.446295,71.036509,78.063231
2,34.816948,14.823514,0.000000,19.752549,33.543568,53.325083,71.915296,78.942019
3,38.867196,18.873762,19.752549,0.000000,13.791019,33.572534,52.162747,59.189470
4,52.658215,32.664781,33.543568,13.791019,0.000000,19.781514,38.371728,45.398450
5,72.439730,52.446295,53.325083,33.572534,19.781514,0.000000,18.590213,25.616936
6,91.029943,71.036509,71.915296,52.162747,38.371728,18.590213,0.000000,7.026723
7,98.056666,78.063231,78.942019,59.189470,45.398450,25.616936,7.026723,0.000000


For clarity : The below table 

*   val[i][j] = -1  means the drone flies directly
*   val[i][j] = k  means the drone flies by stopping at k between i and j
*   val[i][j] = -inf  same location



In [48]:
pd.DataFrame(routePassage)

,0,1,2,3,4,5,6,7
0,-inf,-1.0,1.0,1.0,3.0,4.0,5.0,6.0
1,-1.0,-inf,-1.0,-1.0,3.0,4.0,5.0,6.0
2,1.0,-1.0,-inf,-1.0,3.0,4.0,5.0,6.0
3,1.0,-1.0,-1.0,-inf,-1.0,4.0,5.0,6.0
4,3.0,3.0,3.0,-1.0,-inf,-1.0,5.0,6.0
5,4.0,4.0,4.0,4.0,-1.0,-inf,-1.0,6.0
6,5.0,5.0,5.0,5.0,5.0,-1.0,-inf,-1.0
7,6.0,6.0,6.0,6.0,6.0,6.0,-1.0,-inf


# Part 3: Get drones from Stockholm to the docks!

Let's say we installed empty docks into each of the selected lighthouses. How long would it take to fill every dock if we are dispatching the drones from Stockholm, and a drone takes 45 minutes to fully charge its battery in the dock?

You may choose to write code here at any level of depth for this problem, or you can provide a ballpark estimate with a few written sentences.

![](https://drive.google.com/uc?export=view&id=1WbGVXfwI_K2A_bnJwJATGpzvsH58fQBH)


We see that we have to send 7 drones from Stockholm, the above picture shows the connections.

At each lighthouse the drone will spend 45 mins to charge on its way to the end of the line.

* L1 : All the 7 drones will pass here, we have to consider 6 charging time
* L2 : Only one drone reaches here, and no charging to consider
* L3 : 5 drones passes here and 4 drone stops for charging.
* L4 : 4 drones passes here and 3 drone stops for charging.
* L5 : 3 drones passes here and 2 drone stops for charging.
* L6 : 2 drones passes here and 1 drone stops for charging.
* L7 : Only one drone reaches here, and no charging to consider


And to make things efficient we start a drone from stockholm every 17th minute after releasing the first one. So that the travel time from second drone onwards comes under the charging time of the previous drone at lighthouse 1.



Further on the configurations are here such that we can make sure that a drone reaches the lighthouse, when the previous drone is fully charged and ready go to next lighthouse

![](https://drive.google.com/uc?export=view&id=1QfyMt1cH61-_y5FK3YltlapjN9Cdxw2j)


The above image shows two ways of calculating the time to install all the lighthouses with drones.
First Calculation it takes **361** mins and in the Second calculation it takes **406** mins, thats because in the first scenario when we are flying the drone to closest lighthouse in a different stem(LH2), there are other drone flying consecutively in the bottom path to Toro as shown in the figure. 

**361** minutes

We can come up with a generic algorithm where we send drones to the longest sub-tree first and follow a descending order.
This way we can overlap flying time of multiple drones

# Discussion

Describe roughly the amount of time you spent on this challenge and any notes you would like to share about your approach.

* Time Spend 15 hrs across 4 days.
* Initial challenges were to understand the pydeck layouts and plot the map.
* Initially my thought process was to find a path from Stockholm to Vindo and then to Toro and then back to Stockholm. Even though the initial thought process was little off the track but it did not affect my visualization of the lighthouses.
* As the drone has a flight time of 30 mins I was trying to develop a generic usecase to figure out the bounding boxes inside which we need to find the lighthouses. Even though in the given scenario it dosent effect that much, but if we want to move the project to differnt location this will be useful.
* Calculating the optimal path, required a DP solution and to traceback on how to reach the destination required some manual tracing to see how it flow.
* Finding the time to setup the whole drone system (PartC), it was hard to find the correlation on how taking a particular part is faster. But later I was able to figure out if we send the drone to the longest path first, the time for the drone to reach the final spot would be an approx time to setup the whole system in **most cases**.


# Stretch goal (optional)

Analyze and visualize any other interesting aspect of this proposal. What is some problem we might run into or information the customer could want to know about using this network of docked drones for search and rescue?



*   We will have to build a system in the docking station to store any drone if gone bad, and this system should be connected to the docking station.
  
  If suppose a drone, dosent respond or stops working when docked we need a mechanism to dispose it without manual intervention, so that the new incoming drone can take its place.
*   The customer would probably want to know about the coverage map of all the drones combined, so that during search and rescue, the drone warns when it flies off the region and cant make it back to the docking station.

*   The battery, flight time are all considered here as an ideal time, this may vary based on the environment and usage of the drone, this also should be processed during the drone flight and the coverage map should be updated based on that.

